<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/PFG_RNN_Predictiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.9 MB/s eta 0:00:00


In [4]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt
import unicodedata

In [38]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/RNN'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

['df_8000_rows.xlsx', 'PROVINCIA_MUNICIPIO.xlsx', 'RN_AT.xlsx', 'RN_ACT.xlsx', 'RN_LOG.xlsx', 'RN_EM.xlsx', 'RN_SAAF.xlsx', 'RN_SO.xlsx']


In [39]:
dic_dataframes = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

# Imprimir todas las claves
for clave in dic_dataframes.keys():
    print(clave)

df_8000_rows.xlsx
PROVINCIA_MUNICIPIO.xlsx
RN_AT.xlsx
RN_ACT.xlsx
RN_LOG.xlsx
RN_EM.xlsx
RN_SAAF.xlsx
RN_SO.xlsx


In [40]:
RN_SAAF = dic_dataframes.get("RN_SAAF.xlsx")
RN_AT = dic_dataframes.get("RN_AT.xlsx")
RN_EM = dic_dataframes.get("RN_EM.xlsx")
RN_SO = dic_dataframes.get("RN_SO.xlsx")
RN_LOG = dic_dataframes.get("RN_LOG.xlsx")
df_8000rows = dic_dataframes.get("df_8000_rows.xlsx")
provincia_municipio = dic_dataframes.get('PROVINCIA_MUNICIPIO.xlsx')

In [7]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
# RN_SAAF = pd.read_excel('/content/RN_SAAF.xlsx')
# RN_AT = pd.read_excel('/content/RN_AT.xlsx')
# # RN_ACT = pd.read_excel('/content/RN_ACT.xlsx')
# RN_EM = pd.read_excel('/content/RN_EM.xlsx')
# RN_LOG = pd.read_excel('/content/RN_LOG.xlsx')
# RN_SO = pd.read_excel('/content/RN_SO.xlsx')
# df_8000rows = pd.read_excel('/content/df_8000_rows.xlsx')
# provincia_municipio = pd.read_excel('/content/PROVINCIA_MUNICIPIO.xlsx')

In [41]:
# Convert all string columns to uppercase and remove accents/diacritics in combined_df
df_8000rows = df_8000rows.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)
provincia_municipio = provincia_municipio.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

In [42]:
# Function to add PROVINCIA column based on LOCALIDAD
def add_provincia_column(df, provincia_mapping):
    df['PROVINCIA'] = df['LOCALIDAD'].map(provincia_mapping)
    return df

# Create a dictionary mapping from LOCALIDAD to PROVINCIA
localidad_to_provincia = provincia_municipio.set_index('MUNICIPIO')['PROVINCIA'].to_dict()

# List of RN dataframes to process
rn_dataframes = {'AT': RN_AT, 'EM': RN_EM, 'LOG': RN_LOG, 'SAAF': RN_SAAF, 'SO': RN_SO}

# Add the PROVINCIA column to each RN dataframe
rn_dfs_with_provincia = {name: add_provincia_column(df, localidad_to_provincia) for name, df in rn_dataframes.items()}

# Drop rows with null values in the 'PROVINCIA' column in each RN dataframe
rn_dfs_cleaned = {name: df.dropna(subset=['PROVINCIA']) for name, df in rn_dfs_with_provincia.items()}

In [43]:
# List of columns to keep
required_columns = ['LOCALIDAD', 'GENERO', 'YEAR NACIMIENTO', 'PROVINCIA']

# Function to ensure the dataframe has all required columns
def ensure_columns(df, columns):
    for column in columns:
        if column not in df.columns:
            df[column] = np.nan  # Create column with NaN values if it does not exist
    return df[columns]  # Return dataframe with only the required columns

# Ensure all dataframes have the required columns
rn_dfs_final = {name: ensure_columns(df, required_columns) for name, df in rn_dfs_cleaned.items()}

<ipython-input-43-605fc2068a84>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.nan  # Create column with NaN values if it does not exist


In [44]:
# Mapping dictionary for normalizing gender values
gender_mapping = {
    'FEMENINO': 'MUJER',
    'M': 'MUJER',
    'MASCULINO': 'HOMBRE',
    'H': 'HOMBRE'
}

# Function to normalize the 'GENERO' column in a dataframe
def normalize_gender_column(df):
    df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))
    return df

# Apply the normalization to the 'SO' dataframe
rn_dfs_final['SO'] = normalize_gender_column(rn_dfs_final['SO'])

<ipython-input-44-333b05a14062>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))


In [45]:
# Function to insert random registers into a dataframe where the 'GENERO' column is empty
def insert_random_registers(df, gender_distribution):
    # Find the indices where 'GENERO' is empty
    empty_indices = df[df['GENERO'].isna()].index

    # Generate random genders based on the given distribution for the empty indices
    genders = np.random.choice(
        ['MUJER', 'HOMBRE'],
        size=len(empty_indices),
        p=[gender_distribution['MUJER'], gender_distribution['HOMBRE']]
    )

    # Assign these random genders to the empty 'GENERO' rows
    df.loc[empty_indices, 'GENERO'] = genders
    return df

# Example of usage
gender_distribution = {'MUJER': 0.44, 'HOMBRE': 0.56}

# Insert random registers into a specific dataframe, e.g., 'ACT', where 'GENERO' is empty
rn_dfs_final['SAAF'] = insert_random_registers(rn_dfs_final['SAAF'], gender_distribution)
rn_dfs_final['SO'] = insert_random_registers(rn_dfs_final['SO'], gender_distribution)

In [46]:
# Get the min and max year in the 'YEAR NACIMIENTO' column
min_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].min())
max_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].max())

# Function to generate random years with the specified distribution
def generate_random_years(size):
    # 75% of the years under 2004
    under_2004 = np.random.randint(min_year, 2004, size=int(size * 0.75))
    # 25% of the years between 2004 and max_year
    over_2004 = np.random.randint(2004, max_year + 1, size=size - int(size * 0.75))
    return np.concatenate([under_2004, over_2004])

# Function to insert random years into the 'YEAR NACIMIENTO' column where it is empty
def insert_random_years(df):
    # Find the indices where 'YEAR NACIMIENTO' is empty
    empty_indices = df[df['YEAR NACIMIENTO'].isna()].index

    # Generate random years based on the specified distribution
    random_years = generate_random_years(len(empty_indices))
    np.random.shuffle(random_years)  # Shuffle to mix the values

    # Assign these random years to the empty 'YEAR NACIMIENTO' rows
    df.loc[empty_indices, 'YEAR NACIMIENTO'] = random_years
    return df

# Apply the function to the 'SO' dataframe
rn_dfs_final['SO'] = insert_random_years(rn_dfs_final['SO'])

In [47]:
# Add a column named 'SERVICIO' with the name of the dataframe to each dataframe in rn_dfs_final
def add_servicio_column(dfs):
    for name, df in dfs.items():
        df['SERVICIO'] = name
    return dfs

# Apply the function to rn_dfs_final
rn_dfs_final = add_servicio_column(rn_dfs_final)

<ipython-input-47-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-47-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-47-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [48]:
# Combine all dataframes into a single dataframe
combined_df = pd.concat(rn_dfs_final.values(), ignore_index=True)

# Shuffle the rows of the combined dataframe
shuffled_combined_df = combined_df.sample(frac=1).reset_index(drop=True)

shuffled_combined_df['SERVICIO'].value_counts()

SERVICIO
SO      1457
SAAF     567
EM       203
LOG      182
AT       143
Name: count, dtype: int64

In [49]:
# Crear una función para categorizar por grupos de edad
def categorize_age(year):
    if year < 1940:
        return 'ANTES DE 1940'
    elif year < 1960:
        return '1940-1959'
    elif year < 1980:
        return '1960-1979'
    elif year < 2000:
        return '1980-1999'
    elif year < 2020:
        return '2000-2020'
    else:
        return '2020 EN ADELANTE'

rna1 = shuffled_combined_df.copy()
# Aplicar la función a la columna 'YEAR NACIMIENTO'
rna1['YEAR NACIMIENTO_CAT'] = rna1['YEAR NACIMIENTO'].apply(categorize_age)

# Ver el resultado
print(rna1)

                 LOCALIDAD  GENERO  YEAR NACIMIENTO    PROVINCIA SERVICIO  \
0                 ALBACETE   MUJER           2016.0     ALBACETE     SAAF   
1                   TOLEDO   MUJER           1970.0       TOLEDO       SO   
2              GUADALAJARA   MUJER           1954.0  GUADALAJARA      LOG   
3                 NAMBROCA  HOMBRE           1953.0       TOLEDO       SO   
4     SAN CARLOS DEL VALLE   MUJER           2019.0  CIUDAD REAL       SO   
...                    ...     ...              ...          ...      ...   
2547   AZUQUECA DE HENARES   MUJER           1972.0  GUADALAJARA       SO   
2548           GUADALAJARA  HOMBRE           2016.0  GUADALAJARA       SO   
2549             LOMINCHAR  HOMBRE           2019.0       TOLEDO       AT   
2550                TOLEDO  HOMBRE           1968.0       TOLEDO       EM   
2551            VALDEPENAS  HOMBRE           1956.0  CIUDAD REAL       SO   

     YEAR NACIMIENTO_CAT  
0              2000-2020  
1              1960-1

#Análisis descriptivo de los datos
Análisis descriptivo de los datos por servicio, vamos a ver cómo se distribuyen y qué peso tiene cada servicio en la totalidad de CLM.

**OJOOOOO** SE DEBE COMENTAR LA PARTE DEL ANÁLISIS DESCRIPTIVO PARA EJECUTAR EL CÓDIGO DE LAS RNAS.

In [50]:
df['YEAR NACIMIENTO'].unique()

array([  nan, 1980., 2003., 2007., 1976., 1972., 2010., 1947., 1977.,
       2011., 1981., 1979., 2015., 1959., 1966., 2014., 1974., 2006.,
       2005., 1969., 1968., 1971., 1975., 2013., 1983., 2012., 2016.,
       1996., 1989., 1978., 1999., 2004., 2017., 1984., 1987., 2018.,
       1954., 1995., 1991., 2008., 1955., 1952., 1951., 1957., 1958.,
       1964., 1953., 1956., 2020., 1961., 1988., 2009., 1982., 1985.,
       2021., 1970., 1998., 2002., 1943., 1967., 2000., 1950., 1944.,
       1965., 1935., 1992., 1986., 1997., 2001., 1994., 1949., 1990.,
       2019., 1993., 1973., 1960., 1963., 1929., 1945., 1939., 1962.,
       1948.])

In [52]:
df = rna1.copy()


# Función para calcular la moda con manejo de posibles errores
def mode(series):
    if len(series) == 0:
        return np.nan
    return series.mode().iloc[0]

# Calcular las medidas de tendencia central y dispersión por servicio
descriptive_stats = df.groupby('SERVICIO')['YEAR NACIMIENTO'].agg(['mean', 'median', mode, 'var', 'std', 'min', 'max'])
descriptive_stats['range'] = descriptive_stats['max'] - descriptive_stats['min']

# Mostrar los resultados
display(descriptive_stats)

# Tablas de contingencia
# Contingencia de GENERO por SERVICIO
contingency_gender_service = pd.crosstab(df['SERVICIO'], df['GENERO'])

# Contingencia de PROVINCIA por SERVICIO
contingency_provincia_service = pd.crosstab(df['SERVICIO'], df['PROVINCIA'])

# Contingencia de PROVINCIA por SERVICIO
contingency_year_service = pd.crosstab(df['SERVICIO'], df['YEAR NACIMIENTO_CAT'])

# Mostrar las tablas de contingencia
print("\nContingencia de Genero por Servicio:")
display(contingency_gender_service)

print("\nContingencia de Provincia por Servicio:")
display(contingency_provincia_service)

print("\nContingencia de Año por Servicio:")
display(contingency_year_service)

,mean,median,mode,var,std,min,max,range
SERVICIO,,,,,,,,
AT,2019.475524,2020.0,2020.0,2.237073,1.495685,2016.0,2023.0,7.0
EM,1980.800000,1979.0,1973.0,146.629787,12.109079,1957.0,2003.0,46.0
LOG,2003.804469,2010.0,2015.0,322.236834,17.950956,1941.0,2020.0,79.0
SAAF,1994.918871,2001.0,2016.0,461.890933,21.491648,1905.0,2024.0,119.0
SO,1982.035003,1981.0,2007.0,600.434900,24.503773,1929.0,2021.0,92.0



Contingencia de Genero por Servicio:


GENERO,HOMBRE,MUJER
SERVICIO,,
AT,93,50
EM,102,101
LOG,106,76
SAAF,317,250
SO,736,721



Contingencia de Provincia por Servicio:


PROVINCIA,ALBACETE,CIUDAD REAL,CUENCA,GUADALAJARA,TOLEDO
SERVICIO,,,,,
AT,47,35,8,0,53
EM,27,98,10,2,66
LOG,0,0,35,61,86
SAAF,114,151,73,138,91
SO,252,70,67,650,418



Contingencia de Año por Servicio:


YEAR NACIMIENTO_CAT,1940-1959,1960-1979,1980-1999,2000-2020,2020 EN ADELANTE,ANTES DE 1940
SERVICIO,,,,,,
AT,0,0,0,66,77,0
EM,2,48,38,7,108,0
LOG,10,13,14,138,7,0
SAAF,49,91,129,260,35,3
SO,234,386,306,434,19,78


##RNA predictiva con los datos reales
Vamos a realizar una primera red neuronal con los registros reales.

####Preparación de los datos

Convertimos las variables categóricas en variables dummy usando pd.get_dummies.

Separamos las características (x) de la etiqueta (y).
Convertimos la etiqueta en variables dummy para la clasificación multiclase.

In [17]:
# Convertir las variables categóricas en variables dummy
df = pd.get_dummies(rna1, columns=['LOCALIDAD', 'GENERO', 'PROVINCIA', 'YEAR NACIMIENTO'])

# Separar las características (X) y la etiqueta (y)
X = df.drop('SERVICIO', axis=1)
y = df['SERVICIO']

# Convertir la etiqueta en variables dummy
y = pd.get_dummies(y)

####División de los datos
Usaremos un 70% de los datos para entrenar y del 30% restante, un 70% para validar (21% del total) e ir ajustandio los hiperparámetros (evitando el sobreentrenamiento) y un 30% para el conjunto de test (9% del total), para evaluar el modelo y su precisión al final.

In [18]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# Verificar las formas de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (1786, 228)
Tamaño del conjunto de validación: (536, 228)
Tamaño del conjunto de prueba: (230, 228)


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Definir el modelo
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.4440 - loss: 1.3873 - val_accuracy: 0.5690 - val_loss: 1.1461
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5670 - loss: 1.1304 - val_accuracy: 0.5709 - val_loss: 1.0815
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5781 - loss: 1.0448 - val_accuracy: 0.6138 - val_loss: 1.0391
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6012 - loss: 1.0067 - val_accuracy: 0.6269 - val_loss: 0.9972
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6341 - loss: 0.9549 - val_accuracy: 0.6231 - val_loss: 0.9816
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6457 - loss: 0.9263 - val_accuracy: 0.6269 - val_loss: 0.9582
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6312 - loss: 0.9142 - val_accuracy: 0.6250 - val_loss: 0.9281
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6517 - loss: 0.8891 - val_accuracy: 0.6269 - val_loss

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu, tanh

# Definir el modelo con hiperparámetros ajustables
def create_model(learning_rate=0.001, activation='relu', dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation=activation))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_model(learning_rate=0.0005, activation='tanh', dropout_rate=0.3)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.3855 - loss: 1.4837 - val_accuracy: 0.5970 - val_loss: 1.1657
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6103 - loss: 1.1104 - val_accuracy: 0.6175 - val_loss: 1.0666
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6290 - loss: 1.0314 - val_accuracy: 0.6381 - val_loss: 1.0266
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6459 - loss: 0.9862 - val_accuracy: 0.6343 - val_loss: 0.9912
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6440 - loss: 0.9420 - val_accuracy: 0.6493 - val_loss: 0.9694
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6623 - loss: 0.9116 - val_accuracy: 0.6474 - val_loss: 0.9539
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6592 - loss: 0.9262 - val_accuracy: 0.6493 - val_loss: 0.9488
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6652 - loss: 0.8698 - val_accuracy: 0.6530 - val_loss: 0

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Definir el modelo con hiperparámetros ajustables
def create_simple_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_simple_model(learning_rate=0.001, dropout_rate=0.3)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4483 - loss: 1.5020 - val_accuracy: 0.5690 - val_loss: 1.1758
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5889 - loss: 1.1165 - val_accuracy: 0.5690 - val_loss: 1.1052
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5479 - loss: 1.1106 - val_accuracy: 0.5690 - val_loss: 1.0639
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5869 - loss: 1.0445 - val_accuracy: 0.6063 - val_loss: 1.0280
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5903 - loss: 1.0037 - val_accuracy: 0.6269 - val_loss: 0.9935
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6426 - loss: 0.9355 - val_accuracy: 0.6306 - val_loss: 0.9822
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6284 - loss: 0.9434 - val_accuracy: 0.6306 - val_loss: 0.9570
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6561 - loss: 0.9117 - val_accuracy: 0.6325 - val_loss: 0.9366
Epo

###Modelo XGBoost

In [22]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Codificar las etiquetas usando LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.idxmax(axis=1))  # Convertir de one-hot encoding a etiquetas enteras

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Crear y entrenar el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))
xgb_model.fit(X_train_xgb, y_train_xgb)

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = xgb_model.predict_proba(X_train_xgb)
X_test_nn = xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Definir el modelo de red neuronal
def create_nn_model(input_dim, learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train_nn.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo de red neuronal
nn_model = create_nn_model(input_dim=X_train_nn.shape[1])

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo de red neuronal
history = nn_model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = nn_model.evaluate(X_test_nn, y_test_nn)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5541 - loss: 1.4285 - val_accuracy: 0.5866 - val_loss: 1.0722
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5793 - loss: 1.0606 - val_accuracy: 0.6341 - val_loss: 0.9438
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6220 - loss: 0.9848 - val_accuracy: 0.7235 - val_loss: 0.8784
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6875 - loss: 0.8782 - val_accuracy: 0.7067 - val_loss: 0.8200
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6969 - loss: 0.8506 - val_accuracy: 0.7011 - val_loss: 0.7767
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7096 - loss: 0.7782 - val_accuracy: 0.7179 - val_loss: 0.7580
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7154 - loss: 0.7774 - val_accuracy: 0.7179 - val_loss: 0.7462
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7081 - loss: 0.7920 - val_accuracy: 0.7011 - val_loss: 0.

In [23]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Original Servicio: SO
Servicio: AT, Probabilidad: 0.0001
Servicio: EM, Probabilidad: 0.0036
Servicio: LOG, Probabilidad: 0.0038
Servicio: SAAF, Probabilidad: 0.2018
Servicio: SO, Probabilidad: 0.7907


In [24]:
import numpy as np

# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[22]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[22]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Original Servicio: SO
Servicio: AT, Probabilidad: 0.0001
Servicio: EM, Probabilidad: 0.0035
Servicio: LOG, Probabilidad: 0.0042
Servicio: SAAF, Probabilidad: 0.1932
Servicio: SO, Probabilidad: 0.7990


In [25]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[70]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[70]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Original Servicio: EM
Servicio: AT, Probabilidad: 0.0400
Servicio: EM, Probabilidad: 0.8981
Servicio: LOG, Probabilidad: 0.0008
Servicio: SAAF, Probabilidad: 0.0416
Servicio: SO, Probabilidad: 0.0195


##RNA predictiva con datos generados
Red Neuronal con 8000 registros más que han sido creados teniendo en cuenta las estadísticas reales de los datos, es decir, contemplando cómo se distribuyen por edad, género y localidad y, obviamente, el municipio y la provincia correctamente computados.

Esto se hace para intentar aumentar el porcentaje de precisión obtenido.

In [26]:
# Calculate the frequency of each element in the 'SERVICIO' column
service_frequency = df['SERVICIO'].value_counts()

# Group by 'PROVINCIA' and 'SERVICIO' and count the occurrences
service_province_freq = shuffled_combined_df.groupby(['PROVINCIA', 'SERVICIO']).size().reset_index(name='COUNT')

# Function to assign services based on frequency
def assign_service(row, freq_df):
    province = row['PROVINCIA']
    possible_services = freq_df[freq_df['PROVINCIA'] == province]
    if possible_services.empty:
        return np.nan
    else:
        return np.random.choice(possible_services['SERVICIO'].values, p=possible_services['COUNT'].values / possible_services['COUNT'].sum())

# Apply the function to assign services
df_8000rows['SERVICIO'] = df_8000rows.apply(assign_service, axis=1, args=(service_province_freq,))

# Ensure balanced classes by sampling
min_class_size = service_frequency.min()
balanced_df = df_8000rows.groupby('SERVICIO').apply(lambda x: x.sample(n=min_class_size, replace=True)).reset_index(drop=True)

In [27]:
# Calculate the frequency of each service
service_frequency = shuffled_combined_df['SERVICIO'].value_counts()

# Set the target number of records per service
target_size = 1457

# Create a balanced dataframe starting with the initial shuffled data
to_use_df = shuffled_combined_df.copy()

# Loop through each service to balance the dataframe
for service, count in service_frequency.items():
    if count < target_size:
        # Get the rows for the current service
        service_df = balanced_df[balanced_df['SERVICIO'] == service]
        # Sample additional rows with replacement to reach the target size
        additional_samples = service_df.sample(n=target_size - count, replace=True)
        to_use_df = pd.concat([to_use_df, additional_samples])

# Shuffle the final balanced dataframe
to_use_df = to_use_df.sample(frac=1).reset_index(drop=True)

In [28]:
# Shuffle the rows of the dataframe ready to use
to_use_df = to_use_df.sample(frac=1).reset_index(drop=True)

In [29]:
# Get the min and max year in the 'YEAR NACIMIENTO' column
min_year = int(to_use_df['YEAR NACIMIENTO'].min())
max_year = int(to_use_df['YEAR NACIMIENTO'].max())

# Function to generate random years with the specified distribution
def generate_random_years(size):
    return np.random.randint(min_year, max_year + 1, size=int(size))

# Function to insert random years into the 'YEAR NACIMIENTO' column where it is empty
def insert_random_years(df):
    # Find the indices where 'YEAR NACIMIENTO' is empty
    empty_indices = df[df['YEAR NACIMIENTO'].isna()].index

    # Generate random years based on the specified distribution
    random_years = generate_random_years(len(empty_indices))

    # Assign these random years to the empty 'YEAR NACIMIENTO' rows
    df.loc[empty_indices, 'YEAR NACIMIENTO'] = random_years
    return df

# Apply the function to the dataframe
to_use_df = insert_random_years(to_use_df)

In [30]:
# Crear una función para categorizar por grupos de edad
def categorize_age(year):
    if year < 1940:
        return 'Antes de 1940'
    elif year < 1960:
        return '1940-1959'
    elif year < 1980:
        return '1960-1979'
    elif year < 2000:
        return '1980-1999'
    else:
        return '2000 en adelante'

# Aplicar la función a la columna 'YEAR NACIMIENTO'
to_use_df['YEAR NACIMIENTO'] = to_use_df['YEAR NACIMIENTO'].apply(categorize_age)

# Ver el resultado
print(to_use_df.head())

             LOCALIDAD  GENERO   YEAR NACIMIENTO    PROVINCIA SERVICIO  \
0          GUADALAJARA  HOMBRE         1940-1959  GUADALAJARA       SO   
1            FUENLLANA   MUJER         1940-1959  CIUDAD REAL       EM   
2             GABALDON   MUJER  2000 en adelante       CUENCA       EM   
3                UCEDA  HOMBRE  2000 en adelante  GUADALAJARA     SAAF   
4  AZUQUECA DE HENARES  HOMBRE         1960-1979  GUADALAJARA     SAAF   

     TIPO SORDERA  
0             NaN  
1           MIXTA  
2  NEUROSENSORIAL  
3  NEUROSENSORIAL  
4             NaN  


In [31]:
# Determina la cantidad de servicios únicos
unique_services = to_use_df['SERVICIO'].unique()
num_services = len(unique_services)

# Determina cuántos registros por servicio necesitas para el conjunto de prueba
test_size_per_service = 85 // num_services

# Si hay más de 85 registros, ajusta la muestra para que esté balanceada
if test_size_per_service * num_services < 85:
    additional_samples = 85 - (test_size_per_service * num_services)
else:
    additional_samples = 0

# Crear el conjunto de prueba balanceado
test_df = pd.DataFrame()

for service in unique_services:
    service_df = to_use_df[to_use_df['SERVICIO'] == service]
    test_samples = service_df.sample(n=test_size_per_service, replace=False)
    test_df = pd.concat([test_df, test_samples])

# Añadir muestras adicionales si es necesario
if additional_samples > 0:
    additional_df = to_use_df[~to_use_df.index.isin(test_df.index)]
    additional_samples_df = additional_df.sample(n=additional_samples, replace=False)
    test_df = pd.concat([test_df, additional_samples_df])

# Eliminar las filas seleccionadas del dataframe original para crear el conjunto de entrenamiento/validación
to_use_df = to_use_df.drop(test_df.index)

# Verifica el conjunto de prueba
print(test_df['SERVICIO'].value_counts())
print(test_df.shape)

# Verifica el conjunto de entrenamiento/validación
print(to_use_df.shape)

SERVICIO
SO      17
EM      17
SAAF    17
AT      17
LOG     17
Name: count, dtype: int64
(85, 6)
(7200, 6)


In [32]:
to_use_df = to_use_df.drop(columns=['TIPO SORDERA'])

#Red Neuronal Predictiva

Creación de una red neuronal predictiva que sea capaz de clasificar nuevos usuarios con buena precisión en servicio y provincia.

In [33]:
# Convertir las variables categóricas en variables dummy
df = pd.get_dummies(to_use_df, columns=['LOCALIDAD', 'GENERO', 'PROVINCIA', 'YEAR NACIMIENTO'])

# Separar las características (X) y la etiqueta (y)
X = df.drop('SERVICIO', axis=1)
y = df['SERVICIO']

# Convertir la etiqueta en variables dummy
y = pd.get_dummies(y)

In [34]:
# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# Verificar las formas de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (5040, 516)
Tamaño del conjunto de validación: (1512, 516)
Tamaño del conjunto de prueba: (648, 516)


In [35]:
# Definir el modelo
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.2727 - loss: 1.5763 - val_accuracy: 0.4610 - val_loss: 1.3472
Epoch 2/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4302 - loss: 1.3470 - val_accuracy: 0.6058 - val_loss: 1.0630
Epoch 3/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5800 - loss: 1.0809 - val_accuracy: 0.7156 - val_loss: 0.8114
Epoch 4/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6990 - loss: 0.8582 - val_accuracy: 0.7500 - val_loss: 0.6712
Epoch 5/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7500 - loss: 0.7220 - val_accuracy: 0.7738 - val_loss: 0.5954
Epoch 6/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7710 - loss: 0.6455 - val_accuracy: 0.7910 - val_loss: 0.5616
Epoch 7/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7863 - loss: 0.5869 - val_accuracy: 0.7996 - val_loss: 0.5209
Epoch 8/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8228 - loss: 0.5037 - val_accuracy: 0.8148 - val_

In [36]:
# Definir el modelo con hiperparámetros ajustables
def create_model(learning_rate=0.001, activation='relu', dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation=activation))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_model(learning_rate=0.0005, activation='tanh', dropout_rate=0.3)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=90, batch_size=64, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/90


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.2882 - loss: 1.5559 - val_accuracy: 0.4663 - val_loss: 1.3251
Epoch 2/90
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5018 - loss: 1.2618 - val_accuracy: 0.5807 - val_loss: 1.1193
Epoch 3/90
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6103 - loss: 1.0580 - val_accuracy: 0.6680 - val_loss: 0.9155
Epoch 4/90
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7000 - loss: 0.8719 - val_accuracy: 0.7077 - val_loss: 0.7867
Epoch 5/90
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7415 - loss: 0.7235 - val_accuracy: 0.6984 - val_loss: 0.7271
Epoch 6/90
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7438 - loss: 0.6741 - val_accuracy: 0.7050 - val_loss: 0.7169
Epoch 7/90
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7492 - loss: 0.6401 - val_accuracy: 0.7024 - val_loss: 0.6928
Epoch 8/90
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7586 - loss: 0.5976 - val_accuracy: 0.7057 - val_loss: 0.6920
Epo

In [37]:
# Definir el modelo con hiperparámetros ajustables
def create_simple_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_simple_model(learning_rate=0.001, dropout_rate=0.3)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2276 - loss: 1.5935 - val_accuracy: 0.4312 - val_loss: 1.4597
Epoch 2/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4207 - loss: 1.4055 - val_accuracy: 0.6071 - val_loss: 1.0741
Epoch 3/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6044 - loss: 1.0699 - val_accuracy: 0.7116 - val_loss: 0.8269
Epoch 4/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6994 - loss: 0.8329 - val_accuracy: 0.7500 - val_loss: 0.6868
Epoch 5/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7527 - loss: 0.6895 - val_accuracy: 0.7758 - val_loss: 0.6179
Epoch 6/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7799 - loss: 0.6252 - val_accuracy: 0.7837 - val_loss: 0.5736
Epoch 7/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8027 - loss: 0.5502 - val_accuracy: 0.8069 - val_loss: 0.5367
Epoch 8/50
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8215 - loss: 0.5048 - val_accuracy: 0.

Ejemplo de prueba para ver qué tan bien predice.

In [38]:
import numpy as np

# Ejemplo de un registro individual (preprocesado)
# Asegúrate de que el registro individual tenga las mismas características que los datos de entrenamiento y prueba
new_record = X_test.iloc[3]

# Obtener el servicio objetivo original
original_service = y_test.iloc[3].idxmax()

 # Convertir el nuevo registro a un DataFrame
new_record_df = pd.DataFrame([new_record])

# Asegurarse de que tenga las mismas columnas que el DataFrame de entrenamiento
missing_cols = set(X.columns) - set(new_record_df.columns)
for col in missing_cols:
    new_record_df[col] = 0
new_record_df = new_record_df[X.columns]

# Convertir a numpy array
new_record_array = new_record_df.values

# Predecir usando el modelo entrenado
predictions = model.predict(new_record_array)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Imprimir el servicio objetivo original y las probabilidades
print(f"Original Servicio: {original_service}")
# Imprimir las probabilidades
service_labels = y.columns  # Asegúrate de que 'y' tiene las columnas de las categorías de servicio
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Original Servicio: SAAF
Servicio: AT, Probabilidad: 0.0602
Servicio: EM, Probabilidad: 0.0040
Servicio: LOG, Probabilidad: 0.5968
Servicio: SAAF, Probabilidad: 0.1689
Servicio: SO, Probabilidad: 0.1701


In [39]:
import numpy as np

# Ejemplo de un registro individual (preprocesado)
# Asegúrate de que el registro individual tenga las mismas características que los datos de entrenamiento y prueba
new_record = X_test.iloc[22]

# Obtener el servicio objetivo original
original_service = y_test.iloc[22].idxmax()

 # Convertir el nuevo registro a un DataFrame
new_record_df = pd.DataFrame([new_record])

# Asegurarse de que tenga las mismas columnas que el DataFrame de entrenamiento
missing_cols = set(X.columns) - set(new_record_df.columns)
for col in missing_cols:
    new_record_df[col] = 0
new_record_df = new_record_df[X.columns]

# Convertir a numpy array
new_record_array = new_record_df.values

# Predecir usando el modelo entrenado
predictions = model.predict(new_record_array)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Imprimir el servicio objetivo original y las probabilidades
print(f"Original Servicio: {original_service}")
# Imprimir las probabilidades
service_labels = y.columns  # Asegúrate de que 'y' tiene las columnas de las categorías de servicio
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Original Servicio: SAAF
Servicio: AT, Probabilidad: 0.0257
Servicio: EM, Probabilidad: 0.0218
Servicio: LOG, Probabilidad: 0.0342
Servicio: SAAF, Probabilidad: 0.2865
Servicio: SO, Probabilidad: 0.6317


In [40]:
import numpy as np

# Ejemplo de un registro individual (preprocesado)
# Asegúrate de que el registro individual tenga las mismas características que los datos de entrenamiento y prueba
new_record = X_test.iloc[70]

# Obtener el servicio objetivo original
original_service = y_test.iloc[70].idxmax()

 # Convertir el nuevo registro a un DataFrame
new_record_df = pd.DataFrame([new_record])

# Asegurarse de que tenga las mismas columnas que el DataFrame de entrenamiento
missing_cols = set(X.columns) - set(new_record_df.columns)
for col in missing_cols:
    new_record_df[col] = 0
new_record_df = new_record_df[X.columns]

# Convertir a numpy array
new_record_array = new_record_df.values

# Predecir usando el modelo entrenado
predictions = model.predict(new_record_array)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Imprimir el servicio objetivo original y las probabilidades
print(f"Original Servicio: {original_service}")
# Imprimir las probabilidades
service_labels = y.columns  # Asegúrate de que 'y' tiene las columnas de las categorías de servicio
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Original Servicio: EM
Servicio: AT, Probabilidad: 0.0006
Servicio: EM, Probabilidad: 0.9977
Servicio: LOG, Probabilidad: 0.0000
Servicio: SAAF, Probabilidad: 0.0013
Servicio: SO, Probabilidad: 0.0004


In [41]:
pip install scikeras

In [42]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Definir el modelo con hiperparámetros ajustables
def create_nn_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el KerasClassifier con SciKeras
nn_model = KerasClassifier(model=create_nn_model, verbose=0)

# Definir los hiperparámetros para la búsqueda aleatoria
param_dist_nn = {
    'model__learning_rate': uniform(0.0001, 0.001),
    'model__dropout_rate': uniform(0.1, 0.5),
    'epochs': [50, 100, 200],
    'batch_size': [16, 32, 64]
}

# Realizar la búsqueda aleatoria
random_search_nn = RandomizedSearchCV(estimator=nn_model, param_distributions=param_dist_nn, scoring='accuracy', cv=3, n_iter=20, verbose=1, random_state=42)
random_search_nn.fit(X_train, y_train)

# Obtener el mejor modelo
best_nn_model = random_search_nn.best_estimator_

Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

In [43]:
# Evaluar el mejor modelo de red neuronal
best_nn_model_keras = best_nn_model.model_
loss, accuracy = best_nn_model_keras.evaluate(X_test, y_test)
print(f'Best Neural Network Model Loss: {loss}')
print(f'Best Neural Network Model Accuracy: {accuracy}')

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8329 - loss: 0.6027  
Best Neural Network Model Loss: 0.6062937378883362
Best Neural Network Model Accuracy: 0.834876537322998


##Modelo XGBoost con datos generados

In [44]:
# Codificar las etiquetas usando LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.idxmax(axis=1))  # Convertir de one-hot encoding a etiquetas enteras

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Crear y entrenar el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))
xgb_model.fit(X_train_xgb, y_train_xgb)

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = xgb_model.predict_proba(X_train_xgb)
X_test_nn = xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Definir el modelo de red neuronal
def create_nn_model(input_dim, learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train_nn.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo de red neuronal
nn_model = create_nn_model(input_dim=X_train_nn.shape[1])

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo de red neuronal
history = nn_model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = nn_model.evaluate(X_test_nn, y_test_nn)
print(f'Loss: {loss}, Accuracy: {accuracy}')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.4826 - loss: 1.4587 - val_accuracy: 0.8343 - val_loss: 0.7095
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8021 - loss: 0.6905 - val_accuracy: 0.8323 - val_loss: 0.5214
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8042 - loss: 0.6088 - val_accuracy: 0.8333 - val_loss: 0.5033
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7905 - loss: 0.6195 - val_accuracy: 0.8393 - val_loss: 0.5134
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8012 - loss: 0.5853 - val_accuracy: 0.8373 - val_loss: 0.5006
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8105 - loss: 0.5685 - val_accuracy: 0.8363 - val_loss: 0.5008
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7970 - loss: 0.5821 - val_accuracy: 0.8393 - val_loss: 0.5017
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7999 - loss: 0.5559 - val_accuracy: 0.84

In [45]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Original Servicio: SO
Servicio: AT, Probabilidad: 0.2219
Servicio: EM, Probabilidad: 0.1897
Servicio: LOG, Probabilidad: 0.0173
Servicio: SAAF, Probabilidad: 0.3755
Servicio: SO, Probabilidad: 0.1957


In [46]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Original Servicio: SO
Servicio: AT, Probabilidad: 0.2219
Servicio: EM, Probabilidad: 0.1897
Servicio: LOG, Probabilidad: 0.0173
Servicio: SAAF, Probabilidad: 0.3755
Servicio: SO, Probabilidad: 0.1957


In [47]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Original Servicio: SO
Servicio: AT, Probabilidad: 0.2219
Servicio: EM, Probabilidad: 0.1897
Servicio: LOG, Probabilidad: 0.0173
Servicio: SAAF, Probabilidad: 0.3755
Servicio: SO, Probabilidad: 0.1957


In [48]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Definir el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))

# Definir los hiperparámetros para la búsqueda en cuadrícula
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Realizar la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)
grid_search.fit(X_train_xgb, y_train_xgb)

# Obtener el mejor modelo
best_xgb_model = grid_search.best_estimator_

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = best_xgb_model.predict_proba(X_train_xgb)
X_test_nn = best_xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Evaluar el mejor modelo XGBoost
best_xgb_accuracy = grid_search.best_score_
print(f'Best XGBoost Model Accuracy: {best_xgb_accuracy}')

Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best XGBoost Model Accuracy: 0.726984126984127


In [49]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Definir el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))

# Definir los hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3)
}

# Realizar la búsqueda aleatoria
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, scoring='accuracy', cv=3, n_iter=50, verbose=1, random_state=42)
random_search.fit(X_train_xgb, y_train_xgb)

# Obtener el mejor modelo
best_xgb_model = random_search.best_estimator_

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = best_xgb_model.predict_proba(X_train_xgb)
X_test_nn = best_xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Evaluar el mejor modelo XGBoost
best_xgb_accuracy = random_search.best_score_
print(f'Best XGBoost Model Accuracy: {best_xgb_accuracy}')

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best XGBoost Model Accuracy: 0.7841269841269841


In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Definir el modelo de red neuronal con regularización y Dropout
def create_nn_model(input_dim, learning_rate=0.001, dropout_rate=0.3, regularization=0.01):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(Dense(y_train_nn.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo de red neuronal
nn_model = create_nn_model(input_dim=X_train_nn.shape[1], learning_rate=0.001, dropout_rate=0.4, regularization=0.01)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo de red neuronal
history = nn_model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = nn_model.evaluate(X_test_nn, y_test_nn)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.4482 - loss: 2.6455 - val_accuracy: 0.8839 - val_loss: 1.4699
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8529 - loss: 1.3612 - val_accuracy: 0.8819 - val_loss: 0.9326
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8493 - loss: 1.0110 - val_accuracy: 0.8889 - val_loss: 0.7770
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8497 - loss: 0.8750 - val_accuracy: 0.8829 - val_loss: 0.7014
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8592 - loss: 0.7965 - val_accuracy: 0.8889 - val_loss: 0.6598
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8521 - loss: 0.7455 - val_accuracy: 0.8879 - val_loss: 0.6283
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8478 - loss: 0.7440 - val_accuracy: 0.8819 - val_loss: 0.6155
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8548 - loss: 0.7027 - val_accuracy: 0.8869 - val_loss: 0.6

In [51]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[2]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = best_xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[2]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Original Servicio: LOG
Servicio: AT, Probabilidad: 0.0049
Servicio: EM, Probabilidad: 0.0015
Servicio: LOG, Probabilidad: 0.9762
Servicio: SAAF, Probabilidad: 0.0049
Servicio: SO, Probabilidad: 0.0125
